In [12]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.nn.functional as F

In [8]:
transform = transforms.Compose([transforms.CenterCrop(224),transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = dset.CIFAR10('./',train = True, download = True, transform =transform)
testset = dset.CIFAR10('./',train = False, download = True, transform =transform)
trainset.__dict__

Files already downloaded and verified
Files already downloaded and verified


{'class_to_idx': {'airplane': 0,
  'automobile': 1,
  'bird': 2,
  'cat': 3,
  'deer': 4,
  'dog': 5,
  'frog': 6,
  'horse': 7,
  'ship': 8,
  'truck': 9},
 'classes': ['airplane',
  'automobile',
  'bird',
  'cat',
  'deer',
  'dog',
  'frog',
  'horse',
  'ship',
  'truck'],
 'data': array([[[[ 59,  62,  63],
          [ 43,  46,  45],
          [ 50,  48,  43],
          ...,
          [158, 132, 108],
          [152, 125, 102],
          [148, 124, 103]],
 
         [[ 16,  20,  20],
          [  0,   0,   0],
          [ 18,   8,   0],
          ...,
          [123,  88,  55],
          [119,  83,  50],
          [122,  87,  57]],
 
         [[ 25,  24,  21],
          [ 16,   7,   0],
          [ 49,  27,   8],
          ...,
          [118,  84,  50],
          [120,  84,  50],
          [109,  73,  42]],
 
         ...,
 
         [[208, 170,  96],
          [201, 153,  34],
          [198, 161,  26],
          ...,
          [160, 133,  70],
          [ 56,  31,   7],
       

In [10]:
train_loader = DataLoader(trainset, batch_size =32, shuffle = True, num_workers= 2)
test_loader = DataLoader(testset, batch_size =32, shuffle = False, num_workers= 2)

In [11]:
def conv_2_block(in_dim,out_dim):
    model = nn.Sequential(
            nn.Conv2d(in_dim,out_dim,3,padding=1),
            nn.ReLU(),
            nn.Conv2d(out_dim,out_dim,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
            )
    return model

def conv_3_block(in_dim, out_dim):
    model = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, 3, padding =1),
            nn.ReLU(),
            nn.Conv2d(out_dim, out_dim, 3, padding =1),
            nn.ReLU(),
            nn.Conv2d(out_dim, out_dim, 3, padding =1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)             
            )
    return model

In [13]:
class VGGNET(nn.Module):
    
    def __init__(self, base_dim):
        super(VGGNET,self).__init__()
        self.feature = nn.Sequential(
            conv_2_block(3,base_dim),
            conv_2_block(base_dim,2*base_dim),
            conv_3_block(2*base_dim,4*base_dim),
            conv_3_block(4*base_dim,8*base_dim),
            conv_3_block(8*base_dim,8*base_dim)
        )
        self.fc = nn.Sequential(
            nn.Linear(8*base_dim*7*7,100),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(100,20),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(20,10)
        )
        
    def forward(self,x):
        x = self.feature(x)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [63]:
model = VGGNET(base_dim = 64).cuda()

for i in model.named_children():
    print(i)

('feature', Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (2): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (3): Seq

In [64]:
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.002)

In [ ]:
for i in range(100):
    for img,label in train_loader:
        img = Variable(img).cuda()
        label = Variable(label).cuda()
        
        optimizer.zero_grad()
        output = model(img)
        loss = loss_f(output,label)
        loss.backward()
        optimizer.step()
        
    if i %10 ==0:
        print(loss)

tensor(2.3074, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3015, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.2991, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3054, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3008, device='cuda:0', grad_fn=<NllLossBackward>)
tensor(2.3019, device='cuda:0', grad_fn=<NllLossBackward>)


In [66]:
correct = 0
total = 0

for image, label in test_loader:
    image = Variable(image).cuda()
    label = Variable(label).cuda()
    
    output = model(image)
    _,predict = torch.max(output,1)
    total += label.size(0)
    correct += (predict == label).sum()
    
print("accuracy: {}".format(100* correct/total))

accuracy: 10
